#### Importing Necessary Libraries

In [1]:
import numpy as np
import pandas as pd

import json

from scipy.stats import poisson, binom
from tqdm import tqdm

np.random.seed(42)

#### Defining a few Constant Variables

In [2]:
# Changable Features
BAG_CAPACITY = 25
AVG_CUSTOMERS_PER_DAY = 400
CONVERGENCE_THRESHOLD = 0.01

# Taking a bigger cutoff was giving me resource allocation error as the state space was exploding
MAX_INVENTORY = 25 #AVG_CUSTOMERS_PER_DAY // BAG_CAPACITY
MAX_ORDER = 25 #AVG_CUSTOMERS_PER_DAY // BAG_CAPACITY

# Constant Variables
DISCOUNT_FACTOR = 0.95
BAG_COST = 10
PROFIT_PER_CUP = 2
UNSATISFIED_PENALTY = 1
STALE_PROBABILITY = 0.25
NON_STANDARD_PROBABILITY = 0.05

#### Defining our State Space

Our State space is a combination of all possible inventory size and number of customers for the day.
 - Our inventory size is the number of (**Fresh**) coffee bags at the start of the day
 - We the get our State Space Matrix by one-hot encoding the state space

In [3]:
states_space = [(inventory_size, num_customers) for inventory_size in range(MAX_INVENTORY + 1) for num_customers in range(AVG_CUSTOMERS_PER_DAY + 1)]
state_indices = {state: idx for idx, state in enumerate(states_space)}

state_matrix = pd.get_dummies(states_space, dtype=np.int64).values

state_matrix

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int64)

In [4]:
state_matrix.shape

(10426, 10426)

#### Defining our Action Space

Our Action space is how many coffee bags we need to order for the next day based on today's state (day end inventory and number of customers that came in today)

In [5]:
actions = list(range(MAX_ORDER + 1))
print(actions)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]


#### Creating our Reward Vector for each action in the Action Space

Algorithm:
 - For each action in the action space do the following:
    - Calculate the Action Cost (Number of coffee bags ordered $\times$ Cost of a Single Bag)
    - Calculate the number of bags that don't meet the standard (Using Binomial Distribution)
    - Get the Usable Coffee Bags from the total number of coffee bags ordered
    - Now for each state in the state space do the following:
        - Calculate New Inventory $=$ current inventory $+$ Usable Coffee Bags
        - Calculate Cups Sold, Unsatisfied Customers
        - Based on above, Calculate revenue reward
        - Finally subtract the action cost from revenue (This becomes our reward for this state)

In [6]:
action_reward_dict = {action: None for action in actions}

for action in tqdm(actions):
    reward_vector = []
    action_cost = action * BAG_COST

    num_bags_not_standard = np.random.binomial(action, NON_STANDARD_PROBABILITY)
    standard_coffee = action - num_bags_not_standard

    for inventory, customers in states_space:
        new_inventory = inventory + standard_coffee

        cups_sold = min(customers, new_inventory * BAG_CAPACITY)
        unsatisfied = max(0, customers - cups_sold)
        revenue_reward = (PROFIT_PER_CUP * cups_sold) - (UNSATISFIED_PENALTY * unsatisfied)

        reward_vector.append(revenue_reward - action_cost)

    action_reward_dict[action] = np.array(reward_vector)

action_reward_dict

100%|██████████| 26/26 [00:00<00:00, 168.63it/s]


{0: array([  0,  -1,  -2, ..., 796, 798, 800]),
 1: array([-10, -11, -12, ..., 786, 788, 790]),
 2: array([-20, -18, -16, ..., 776, 778, 780]),
 3: array([-30, -28, -26, ..., 766, 768, 770]),
 4: array([-40, -38, -36, ..., 756, 758, 760]),
 5: array([-50, -48, -46, ..., 746, 748, 750]),
 6: array([-60, -58, -56, ..., 736, 738, 740]),
 7: array([-70, -68, -66, ..., 726, 728, 730]),
 8: array([-80, -78, -76, ..., 716, 718, 720]),
 9: array([-90, -88, -86, ..., 706, 708, 710]),
 10: array([-100,  -98,  -96, ...,  696,  698,  700]),
 11: array([-110, -108, -106, ...,  686,  688,  690]),
 12: array([-120, -118, -116, ...,  676,  678,  680]),
 13: array([-130, -128, -126, ...,  666,  668,  670]),
 14: array([-140, -138, -136, ...,  656,  658,  660]),
 15: array([-150, -148, -146, ...,  646,  648,  650]),
 16: array([-160, -158, -156, ...,  636,  638,  640]),
 17: array([-170, -168, -166, ...,  626,  628,  630]),
 18: array([-180, -178, -176, ...,  616,  618,  620]),
 19: array([-190, -188, -

In [7]:
action_reward_dict[0].shape

(10426,)

In [8]:
action_reward_dict_json = {key: value.tolist() for key, value in action_reward_dict.items()}

with open('reward_dict.json', 'w') as file:
    json.dump(action_reward_dict_json, file)

#### Creating Customer Probability Matrix (For Faster Calculations in Transition Probabilities)

In [9]:
customer_probability_matrix = np.zeros((AVG_CUSTOMERS_PER_DAY + 1, AVG_CUSTOMERS_PER_DAY + 1), dtype=np.float64)

for customers_today in range(AVG_CUSTOMERS_PER_DAY + 1):
    customer_probability_matrix[customers_today, :] = poisson.pmf(k=np.arange(AVG_CUSTOMERS_PER_DAY + 1), mu=(100 + (3 / 4) * customers_today))

In [10]:
customer_probability_matrix

array([[3.72007598e-044, 3.72007598e-042, 1.86003799e-040, ...,
        9.27193800e-112, 2.32379399e-112, 5.80948496e-113],
       [1.75723946e-044, 1.77041876e-042, 8.91848451e-041, ...,
        8.57001285e-111, 2.16398194e-111, 5.45052952e-112],
       [8.30061148e-045, 8.42512066e-043, 4.27574873e-041, ...,
        7.74842817e-110, 1.97109138e-110, 5.00164437e-111],
       ...,
       [8.58319465e-174, 3.42040307e-171, 6.81515311e-169, ...,
        1.99867101e-002, 1.99616641e-002, 1.98868078e-002],
       [4.05441407e-174, 1.61872482e-171, 3.23137942e-169, ...,
        1.99538583e-002, 1.99663607e-002, 1.99289238e-002],
       [1.91516960e-174, 7.66067839e-172, 1.53213568e-169, ...,
        1.98931014e-002, 1.99429588e-002, 1.99429588e-002]])

#### Calculating Transition Probabilities

Algorithm:
 - For each action in the action space do the following:
    - Random Probability of Bags not meeting the standard
    - For all state (inventory, customers) in the state space do the following:
        - New Inventory $=$ Old Inventory $+$ Action $-$ Bags not meeting the standard (Using Binomial Distribution)
        - Calculate Leftover Bags
        - Calculate the Stale Probability bags counting from 0 to leftover bags
        - For Each next day customer value do the following:
            - Get the Customer Probability
            - Get next states (After subtracting Leftover with stale bags)
            - Calculate the Probability as: $Customer Probability \times (1 - Stale Probability) \times (1 - Not Standard Probability)$

In [11]:
action_transition_dict = {}

for action in actions:
    not_standard_probability = np.random.choice(binom.pmf(k=list(range(action + 1)), n=action, p=NON_STANDARD_PROBABILITY))
    action_transition_matrix = np.zeros(state_matrix.shape, dtype=np.float32)

    for inventory, customers in tqdm(states_space, desc=f"Building Transition Matrices for Action ({action}): "):
        current_state_index = state_indices[(inventory, customers)]

        new_inventory = inventory + action - np.random.binomial(action, NON_STANDARD_PROBABILITY)

        cups_sold = np.minimum(customers, new_inventory * BAG_CAPACITY)
        bags_used = np.minimum(new_inventory, cups_sold // BAG_CAPACITY)
        leftover = int(new_inventory - bags_used)

        stale_counts = np.arange(leftover + 1)
        stale_probs = binom.pmf(k=stale_counts, n=leftover, p=STALE_PROBABILITY)

        for next_day_customers in range(AVG_CUSTOMERS_PER_DAY + 1):
            customer_prob = customer_probability_matrix[customers, next_day_customers]

            for stale_bags, stale_prob in zip(stale_counts, stale_probs):
                usable_bags = int(leftover - stale_bags)
                next_state = (usable_bags, next_day_customers)

                if next_state in state_indices:
                    next_state_index = state_indices[next_state]
                    action_transition_matrix[current_state_index, next_state_index] += customer_prob * (1 - stale_prob) * (1 - not_standard_probability)

    row_sums = action_transition_matrix.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1
    action_transition_matrix /= row_sums

    action_transition_dict[action] = action_transition_matrix

Building Transition Matrices for Action (25): 100%|██████████| 10426/10426 [01:46<00:00, 97.91it/s]


#### Apply Value Iteration

Same Code as in your Lecture Code, just modified for matrix multiplication

In [12]:
policy = {state: 0 for state in states_space}
value_function = np.zeros(state_matrix.shape[0], dtype=np.float64)

In [13]:
def bellman_backup(state: tuple, policy: dict, value_func: np.array, reward: np.array, transition: np.array):
    action = policy[state]

    reward_value = reward[action].reshape(-1)[state_indices[state]]
    transition_vector = transition[action][state_indices[state]]

    return reward_value + DISCOUNT_FACTOR * np.dot(transition_vector, value_func)

In [14]:
delta = float('inf')
iteration = 0
pbar = tqdm(desc="Iterations (approx.)", leave=True)

while delta > CONVERGENCE_THRESHOLD:
    delta = 0
    Q_values = []
    new_value_function = np.zeros_like(value_function)

    for action in actions:
        reward_vector = action_reward_dict[action].reshape(-1)
        transition_matrix = action_transition_dict[action]

        action_value = reward_vector + DISCOUNT_FACTOR * np.dot(transition_matrix, value_function)
        Q_values.append(action_value)

    new_actions = np.argmax(Q_values, axis=0)

    for state_index, state in enumerate(states_space):
        policy[state] = new_actions[state_index]

    new_value_function = np.array([bellman_backup(
            state=state, policy=policy, value_func=value_function, reward=action_reward_dict, transition=action_transition_dict) for state in states_space])

    delta = np.max(np.abs(new_value_function - value_function))

    value_function = new_value_function.copy()
    iteration += 1

    pbar.set_description(f"Iteration {iteration}, Delta: {delta:.3f}")
    pbar.update(1)

Iteration 216, Delta: 0.010: : 216it [1:17:22, 12.92s/it]

In [15]:
for state in policy:
    if policy[state] > 0:
        print(f"Current Inventory: {state[0]}, Customers Today: {state[1]}, Action for next Day: {policy[state]}")

Current Inventory: 0, Customers Today: 0, Action for next Day: 1
Current Inventory: 0, Customers Today: 1, Action for next Day: 1
Current Inventory: 0, Customers Today: 2, Action for next Day: 2
Current Inventory: 0, Customers Today: 3, Action for next Day: 2
Current Inventory: 0, Customers Today: 4, Action for next Day: 2
Current Inventory: 0, Customers Today: 5, Action for next Day: 2
Current Inventory: 0, Customers Today: 6, Action for next Day: 2
Current Inventory: 0, Customers Today: 7, Action for next Day: 2
Current Inventory: 0, Customers Today: 8, Action for next Day: 2
Current Inventory: 0, Customers Today: 9, Action for next Day: 2
Current Inventory: 0, Customers Today: 10, Action for next Day: 2
Current Inventory: 0, Customers Today: 11, Action for next Day: 2
Current Inventory: 0, Customers Today: 12, Action for next Day: 2
Current Inventory: 0, Customers Today: 13, Action for next Day: 2
Current Inventory: 0, Customers Today: 14, Action for next Day: 2
Current Inventory: 0

In [16]:
policy_json = {str(key): int(value) for key, value in policy.items()}

with open('value_iteration_policy.json', 'w') as file:
    json.dump(policy_json, file)

#### Apply Policy Iterations

Same Code as in your Lecture Code, just modified for matrix multiplication

In [17]:
policy = {state: 0 for state in states_space}
old_policy = None

new_value_function = np.zeros(state_matrix.shape[0], dtype=np.float64)
value_function = new_value_function - 0.1

In [18]:
def bellman_backup(state: tuple, policy: dict, value_func: np.array, reward: np.array, transition: np.array):
    action = policy[state]

    reward_value = reward[action].reshape(-1)[state_indices[state]]
    transition_vector = transition[action][state_indices[state]]

    return reward_value + DISCOUNT_FACTOR * np.dot(transition_vector, value_func)

In [19]:
value_iteration = 0
policy_iteration = 0

while policy != old_policy:
    delta = float('inf')

    while delta > CONVERGENCE_THRESHOLD:
        delta = 0
        new_value_function = np.array([bellman_backup(
            state=state, policy=policy, value_func=value_function, reward=action_reward_dict, transition=action_transition_dict) for state in states_space])

        delta = np.sum(new_value_function - value_function)

        value_function = new_value_function.copy()
        value_iteration += 1

    print(f"Number of Value Iterations: {value_iteration}")

    Q_values = []
    new_policy = {}

    for action in actions:
        reward_vector = action_reward_dict[action].reshape(-1)
        transition_matrix = action_transition_dict[action]

        action_value = reward_vector + DISCOUNT_FACTOR * np.dot(transition_matrix, value_function)
        Q_values.append(action_value)

    new_actions = np.argmax(Q_values, axis=0)
    new_value_function = np.max(Q_values, axis=0)

    for state_index, state in enumerate(states_space):
        new_policy[state] = new_actions[state_index]

    value_function = new_value_function.copy()

    old_policy = policy.copy()
    policy = new_policy

    policy_iteration += 1
    print(f"Number of Policy Iterations: {policy_iteration}")

Number of Value Iterations: 2
Number of Policy Iterations: 1
Number of Value Iterations: 83
Number of Policy Iterations: 2
Number of Value Iterations: 470
Number of Policy Iterations: 3
Number of Value Iterations: 827
Number of Policy Iterations: 4
Number of Value Iterations: 1024
Number of Policy Iterations: 5
Number of Value Iterations: 1025
Number of Policy Iterations: 6


In [20]:
for state in policy:
    if policy[state] > 0:
        print(f"Current Inventory: {state[0]}, Customers Today: {state[1]}, Action for next Day: {policy[state]}")

Current Inventory: 0, Customers Today: 0, Action for next Day: 1
Current Inventory: 0, Customers Today: 1, Action for next Day: 1
Current Inventory: 0, Customers Today: 2, Action for next Day: 2
Current Inventory: 0, Customers Today: 3, Action for next Day: 2
Current Inventory: 0, Customers Today: 4, Action for next Day: 2
Current Inventory: 0, Customers Today: 5, Action for next Day: 2
Current Inventory: 0, Customers Today: 6, Action for next Day: 2
Current Inventory: 0, Customers Today: 7, Action for next Day: 2
Current Inventory: 0, Customers Today: 8, Action for next Day: 2
Current Inventory: 0, Customers Today: 9, Action for next Day: 2
Current Inventory: 0, Customers Today: 10, Action for next Day: 2
Current Inventory: 0, Customers Today: 11, Action for next Day: 2
Current Inventory: 0, Customers Today: 12, Action for next Day: 2
Current Inventory: 0, Customers Today: 13, Action for next Day: 2
Current Inventory: 0, Customers Today: 14, Action for next Day: 2
Current Inventory: 0

In [21]:
policy_json = {str(key): int(value) for key, value in policy.items()}

with open('policy_iteration_policy.json', 'w') as file:
    json.dump(policy_json, file)